## RAG Evaluation Notebook
by Andy Garcia


For this notebook, we will use a sample HR Manual from this source:<br>
https://www.icpau.co.ug/sites/default/files/Resources/SAMPLE%20HR%20MANUAL.pdf<br>
It's already downloaded in the ./data directory.

### 1. RAG
Let's start by learning how to use Retriever Augmented Generation (RAG). There are two ways we can go about this: via the LLM's native API or via an LLM agnostic framework. We will explore using OpenAI's native API and the Langchain framework. For both cases we will use OpenAI as the LLM for generation, so you will need a working OpenAI API key. Set the environment variable using the cell below. Reminder: DO NOT upload your API KEY to Git!!!

In [ ]:
%env OPENAI_API_KEY=<YOUR API KEY HERE>

In [ ]:
import os
# Check if OPENAI_API_KEY is visible to the notebook
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
print(f'API KEY={OPENAI_API_KEY}')

### 1a. OpenAI API

### 1b. Langchain

### 2. LLM Evaluation
There are several ways to evaluate the performance of RAG. Here, we will explore the Langchain Evaluator Framework

### 3. LLM Evaulaion via Confusion Matrix
Now we explore using the well-known Machine Learning method